In [ ]:
# Add the relevant scripts from LArMachineLearningData
# Nice the process so it can run with lots of cores on low priority
import os
os.nice(0)

# Add local paths
import sys
hnlDIR = os.environ['_']
sys.path.append('../pyscript')
from SystematicsHelpers import *

import pickle
import uproot
import pandas as pd


import warnings
warnings.filterwarnings("ignore")

<h1>Configuration Stuff Here</h1>

In [ ]:
m = 240

<h1>HNL</h1>

In [ ]:
hnl_path = "../../pkl_files/v3_April2024/df_postselect_m"+str(m)+"_v3_hnl.pkl"

file = open(hnl_path, 'rb')
df_hnl = pickle.load(file)
file.close
del hnl_path

In [ ]:
df_hnl

In [ ]:
df_hnl = df_hnl[["run", "subrun", "event", "slc_id"
                 , "mod_t", 'slc_comp', 'slc_true_event_type'
                 , "true_counts" , "start_counts"
                 , "scale_pot" , "sim_scale_pot"
                 , "simU", "plotU"
                ]]

In [ ]:
count = 3
if m == 140:
    count = 3
if m == 160:
    count = 3
if m == 180:
    count = 3
if m == 200:
    count = 4
if m == 220:
    count = 3
if m == 240:
    count = 4
print(count)

In [ ]:
dfw_hnl = pd.DataFrame()

print(len(df_hnl))

for i in range(0, count):
    hnl_path = "../../pkl_files/v3_April2024/hnl_m"+str(m)+"_flxw_{}.pkl".format(i)
    print(hnl_path)
    
    file = open(hnl_path, 'rb')   
    df = pickle.load(file)
    
    #keep certain branches only
    branches = ['run','subrun','event','slc_id', 'slc_comp', 'slc_flux_weight_total'] + flux_list
    df = df[branches]
    
 
    df = df.merge(df_hnl, how='inner', on=['run','subrun','event','slc_id','slc_comp'])
    print(len(df))
    
    dfw_hnl = pd.concat((dfw_hnl, df), ignore_index=True)
    file.close()
    
del df_hnl

print(len(dfw_hnl))

In [ ]:
dfw_hnl.to_pickle("../../pkl_files/v3_April2024/df_postselect_weight_m"+str(m)+"_v3_hnl.pkl", protocol = 5)

<h1>Intime Cosmics</h1>

In [ ]:
cos_path = "../../pkl_files/v3_April2024/df_postselect_v3_cos.pkl"

file = open(cos_path, 'rb')
df_cos = pickle.load(file)
file.close
del cos_path

In [ ]:
df_cos = df_cos[["run", "subrun", "event", "slc_id", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]
df_cos

In [ ]:
df_cos.to_pickle("../../pkl_files/v3_April2024/df_postselect_weight_v3_cos.pkl", protocol = 5)

<h1> Neutrino </h1>

In [ ]:
nu_path = "../../pkl_files/v3_April2024/df_postselect_v3_nu.pkl"

file = open(nu_path, 'rb')
df_nu = pickle.load(file)
file.close
del nu_path

In [ ]:
df_nu = df_nu[["run", "subrun", "event", "slc_id", "scale_pot", "mod_t", 'slc_comp', 'slc_true_event_type']]

<h3>Rockbox</h3>

In [ ]:
dfw_nu = pd.DataFrame()

print(len(df_nu))

for i in range(0, 51):
    nu_path = "../../pkl_files/v3_April2024/rockbox_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    #print(df.columns)
    
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_id', 'slc_comp'])
    print(len(df))
    
    dfw_nu = pd.concat((dfw_nu, df), ignore_index=True)
    file.close()

print(len(dfw_nu))

<h3> NCPi0 </h3>

In [ ]:
dfw_ncpi0 = pd.DataFrame()

print(len(df_nu))

for i in range(0, 15):
    nu_path = "../../pkl_files/v3_April2024/ncpi0_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    #print(df.columns)
    
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_id', 'slc_comp'])
    print(len(df))
    
    dfw_ncpi0 = pd.concat((dfw_ncpi0, df), ignore_index=True)
    file.close()

print(len(dfw_ncpi0))

<h3> CCNue </h3>

In [ ]:
dfw_ccnue = pd.DataFrame()

print(len(df_nu))

for i in range(0, 4):
    nu_path = "../../pkl_files/v3_April2024/ccnue_flxw_{}.pkl".format(i)
    print(nu_path)
    
    file = open(nu_path, 'rb')
    df = pickle.load(file)
    
    #print(df.columns)
    
    df = df.merge(df_nu, how='inner', on=['run','subrun','event','slc_id', 'slc_comp'])
    print(len(df))
    
    dfw_ccnue = pd.concat((dfw_ccnue, df), ignore_index=True)
    file.close()
    
print(len(dfw_ccnue))

<h3>Merge</h3>

In [ ]:
dfw_nu_combined = pd.concat([dfw_nu, dfw_ncpi0, dfw_ccnue])
print(len(dfw_nu_combined))



In [ ]:
dfw_nu = dfw_nu_combined

In [ ]:
dfw_nu.to_pickle("../../pkl_files/v3_April2024/df_postselect_weight_v3_nu.pkl", protocol = 5)

In [ ]:
del dfw_nu
del dfw_ncpi0
del dfw_ccnue
del dfw_nu_combined

###### 